In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-10-03 10:05:41--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2025-10-03 10:05:41 (176 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code

In [ ]:

# Deleting books with more than 100 reviews and users with more than 200 reviews.
user_unique= df_ratings["user"].value_counts()
isbn_unique= df_ratings["isbn"].value_counts()
user_unique = user_unique[user_unique >= 200].index
isbn_unique = isbn_unique[isbn_unique >= 100].index
df_ratings = df_ratings.loc[(df_ratings["user"].isin(user_unique.values)) & (df_ratings["isbn"].isin(isbn_unique.values))]

fin_users = df_ratings["user"].unique()

In [ ]:

# Join the tables (inner join)
books = pd.merge(left = df_ratings, right = df_books, how = "inner", on = "isbn")
books.drop_duplicates(subset = ["user", "title"], inplace = True, ignore_index = True)

books = pd.pivot(books, index = ["isbn", "author", "title"], columns = "user", values = "rating")
books.reset_index(inplace = True)

# Impute missing ratings with 0
books[fin_users] = books[fin_users].fillna(0.0)

In [ ]:

# Init and fit model
model = NearestNeighbors(n_neighbors = 6, metric = "cosine", algorithm = "brute", n_jobs = -1)
model = model.fit(books[fin_users])

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book=""):
    if book not in books["title"].to_list():
        raise ValueError("Requested book not in database.")
    # Get the requested book's ratings
    ratings = books.loc[books["title"] == book, fin_users]
    # Get the 6 nearest neighbors
    distances, indexes = model.kneighbors(ratings, return_distance=True)
    # Reversing the order.
    distances = distances[0][-1:0:-1]
    indexes = indexes[0][-1:0:-1]
    recommended_books = [[books.loc[index, "title"], distance]for index, distance in zip(indexes, distances)]
    recommended_books = [book] + [recommended_books]
    return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()